In [4]:
import numpy as np
import pandas as pd
from typing import Mapping, List, Tuple
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_boston, load_iris, load_wine, load_digits, \
    load_breast_cancer, load_diabetes, fetch_mldata
from  matplotlib.collections import LineCollection
import time
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype, is_bool_dtype
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from pdpbox import pdp
from rfpimp import *
from scipy.integrate import cumtrapz
# from stratx.plot import *
# from stratx.ice import *

%config InlineBackend.figure_formats = ['png']

def df_string_to_cat(df:pd.DataFrame) -> dict:
    catencoders = {}
    for colname in df.columns:
        if is_string_dtype(df[colname]) or is_object_dtype(df[colname]):
            df[colname] = df[colname].astype('category').cat.as_ordered()
            catencoders[colname] = df[colname].cat.categories
    return catencoders


def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1

In [7]:
def load_rent():
    """
    *Data use rules prevent us from storing this data in this repo*. Download the data
    set from Kaggle. (You must be a registered Kaggle user and must be logged in.)
    Go to the Kaggle [data page](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/data)
    and save `train.json`
    :return:
    """
    df = pd.read_json('data/train.json')

    # Create ideal numeric data set w/o outliers etc...
    df = df[(df.price > 1_000) & (df.price < 10_000)]
    df = df[df.bathrooms <= 4]  # There's almost no data for above with small sample
    df = df[(df.longitude != 0) | (df.latitude != 0)]
    df = df[(df['latitude'] > 40.55) & (df['latitude'] < 40.94) &
            (df['longitude'] > -74.1) & (df['longitude'] < -73.67)]
    df = df.sort_values('created')
    df_rent = df[['bedrooms', 'bathrooms', 'latitude', 'longitude', 'price']]

    return df_rent

In [24]:
def leaf_samples(rf, X:np.ndarray):
    """
    Return a list of arrays where each array is the set of X sample indexes
    residing in a single leaf of some tree in rf forest.
    """
    ntrees = len(rf.estimators_)
    leaf_ids = rf.apply(X) # which leaf does each X_i go to for each tree?
    d = pd.DataFrame(leaf_ids, columns=[f"tree{i}" for i in range(ntrees)])
    d = d.reset_index() # get 0..n-1 as column called index so we can do groupby
    """
    d looks like:
        index	tree0	tree1	tree2	tree3	tree4
    0	0	    8	    3	    4	    4	    3
    1	1	    8	    3	    4	    4	    3
    """
    leaf_samples = []
    for i in range(ntrees):
        """
        Each groupby gets a list of all X indexes associated with same leaf. 4 leaves would
        get 4 arrays of X indexes; e.g.,
        array([array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
               array([10, 11, 12, 13, 14, 15]), array([16, 17, 18, 19, 20]),
               array([21, 22, 23, 24, 25, 26, 27, 28, 29]), ... )
        """
        sample_idxs_in_leaf = d.groupby(f'tree{i}')['index'].apply(lambda x: x.values)
        leaf_samples.extend(sample_idxs_in_leaf) # add [...sample idxs...] for each leaf
    return leaf_samples

In [81]:
df_rent = load_rent()
df_rent = df_rent[-50:]  # get a small subsample since SVM is slowwww
df_rent.head(3)

,bedrooms,bathrooms,latitude,longitude,price
42550,1,1.0,40.7730,-73.9558,2275
10260,3,1.0,40.6895,-73.9566,2725
7428,2,1.0,40.6774,-73.9487,2575


In [82]:
X = df_rent.drop('price', axis=1)
y = df_rent['price']

In [83]:
colname = 'latitude'
ntrees = 1
min_samples_leaf=8
bootstrap = False
max_features = 1.0
rf = RandomForestRegressor(n_estimators=ntrees,
                           min_samples_leaf=min_samples_leaf,
                           bootstrap = bootstrap,
                           max_features = max_features)
rf.fit(X.drop(colname, axis=1), y)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=1.0, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=8, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [84]:
uniq_x = np.array(sorted(np.unique(X[colname])))
uniq_x[0:10]

array([40.6774, 40.6895, 40.6955, 40.706 , 40.7074, 40.7078, 40.7113,
       40.7156, 40.7184, 40.7197])

In [85]:
leaves = leaf_samples(rf, X.drop(colname, axis=1))
nnodes = rf.estimators_[0].tree_.node_count
print(f"Partitioning 'x not {colname}': {nnodes} nodes in (first) tree, "
      f"{len(rf.estimators_)} trees, {len(leaves)} total leaves")
leaves

Partitioning 'x not latitude': 9 nodes in (first) tree, 1 trees, 5 total leaves


[array([ 7, 14, 16, 30, 36, 37, 38, 40, 47]),
 array([12, 24, 25, 26, 27, 31, 32, 39]),
 array([ 0,  4,  5,  8, 10, 15, 17, 23, 29, 34, 43, 46]),
 array([ 9, 11, 13, 19, 20, 21, 22, 28, 35, 41, 42, 45]),
 array([ 1,  2,  3,  6, 18, 33, 44, 48, 49])]

In [88]:
nbins = 2

for samples in leaves:
    leaf_all_x = X.iloc[samples]
    leaf_x = leaf_all_x[colname].values
    leaf_y = y.iloc[samples].values
    print(leaf_x, '->', leaf_y)
    domain = (np.min(leaf_x), np.max(leaf_x))
    bins = np.linspace(*domain, num=nbins+1, endpoint=True)
    binned_idx = np.digitize(leaf_x, bins)
    point_betas = np.full(shape=(len(X),), fill_value=np.nan)
    for b in range(1, len(bins)+1):
        bin_x = leaf_x[binned_idx == b]
        bin_y = leaf_y[binned_idx == b]
        if len(bin_x)<2: # either no or too little data
#             print(f'ignoring {bin_x} -> {bin_y}')
            continue
        r = (np.min(bin_x), np.max(bin_x))
        if np.isclose(r[0], r[1]):
#             print(f'ignoring {bin_x} -> {bin_y} for same range')
            continue
        lm = LinearRegression()
#         bin_x = bin_x.reshape(-1, 1)
        lm.fit(bin_x.reshape(-1, 1), bin_y)
        print(f'\tbin{b}', bin_x, '->', bin_y, 'beta', lm.coef_[0])
        point_betas[binned_idx == b] = lm.coef_[0]

[40.7663 40.7078 40.7258 40.7512 40.7521 40.777  40.7556 40.7356 40.7597] -> [1850 3575 2275 3820 4415 4065 4095 3850 2650]
	bin1 [40.7078 40.7258 40.7356] -> [3575 2275 3850] beta -268.25833277742396


IndexError: boolean index did not match indexed array along dimension 0; dimension is 50 but corresponding boolean dimension is 9